# Foundations Of AIML
## Session 11
### Experiment 2.1: Siamese networks

In [12]:
# The below are wrapper functions used to connect to your drive and this needs to be run once (i.e. once every new session or possibily refreshes for every 24 hours)

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Authentication for your google drive
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()

# Authentication for the wrapper libraries  or possibily refreshes for every 24 hours)

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse


import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

gpg: keybox '/tmp/tmpddtl5j9g/pubring.gpg' created
gpg: /tmp/tmpddtl5j9g/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
gpg: keybox '/tmp/tmp4771w62r/pubring.gpg' created
gpg: /tmp/tmp4771w62r/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [13]:

!google-drive-ocamlfuse drive

Error: Mountpoint drive should be an existing directory.


In [14]:
!ls

Additional Materials.odt
AIML030118.csv
AIML030118.csv.ods
AIML030118.csv.ods (b5e77e5f)
AIML Batch 2 Print
Aiml -Leaves.ods
AI-ML_Ops (3).xlsx
AI-ML_Ops (3).xlsx.ods
AI-ML_Ops (3).xlsx.ods (8ea6dfc3)
Applications of AIML.odt
CFU -7.odt
CFU-Explanation
CFUs-4.txt
CFUs-4.txt.odt
CFUS.odt
checkpoint
checkpoint_ckpt.t7
cifar-100-python.tar.gz
Colab Notebooks
Collab-docs
Decison Trees
Fully Paid AIML Candidates as per Drupal- 03rd Jan(05-35 PM).xlsx
Fully Paid AIML Candidates as per Drupal- 03rd Jan(05-35 PM).xlsx.ods
Fully Paid AIML Candidates as per Drupal- 03rd Jan(05-35 PM).xlsx.ods (ace43b54)
Getting started
Homework CFU - 4.odt
Instance Issues.ods
light_cnn
neuron.odt
Photo Status.ods
Python Interview Questions.odt
Revision1
Session06-Work
SESSION-07
Session-08
Session-11-Experiments
Session 1 - CFU.odt
Untitled presentation.pdf
Working on Google Colab.pdf


In [15]:
!ls

Additional Materials.odt
AIML030118.csv
AIML030118.csv.ods
AIML030118.csv.ods (b5e77e5f)
AIML Batch 2 Print
Aiml -Leaves.ods
AI-ML_Ops (3).xlsx
AI-ML_Ops (3).xlsx.ods
AI-ML_Ops (3).xlsx.ods (8ea6dfc3)
Applications of AIML.odt
CFU -7.odt
CFU-Explanation
CFUs-4.txt
CFUs-4.txt.odt
CFUS.odt
checkpoint
checkpoint_ckpt.t7
cifar-100-python.tar.gz
Colab Notebooks
Collab-docs
Decison Trees
Fully Paid AIML Candidates as per Drupal- 03rd Jan(05-35 PM).xlsx
Fully Paid AIML Candidates as per Drupal- 03rd Jan(05-35 PM).xlsx.ods
Fully Paid AIML Candidates as per Drupal- 03rd Jan(05-35 PM).xlsx.ods (ace43b54)
Getting started
Homework CFU - 4.odt
Instance Issues.ods
light_cnn
neuron.odt
Photo Status.ods
Python Interview Questions.odt
Revision1
Session06-Work
SESSION-07
Session-08
Session-11-Experiments
Session 1 - CFU.odt
Untitled presentation.pdf
Working on Google Colab.pdf


In [21]:
%cd Session-11-Experiments

/content/drive/Session-11-Experiments


In [0]:
# http://pytorch.org/
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.3.0.post4-{platform}-linux_x86_64.whl torchvision
import torch

In [0]:
# Importing pytorch packages
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
# Importing config.py file
import config as cf
from utils import *
from light_cnn import network_9layers
#from resnet import resnet18
from siamese_data_loader import *
from contrastive import *   ### implementation of contrastive loss
## Importing python packages
import os
import sys
import time
import datetime
import numpy as np
import math
import matplotlib.pyplot as plt

In [23]:
img_root = cf.data_dir+'IMFDB_final/'

train_list_file = cf.data_dir+'IMFDB_train_sorted.txt'   #### 5000 images for training
val_list_file = cf.data_dir+'IMFDB_test_sorted.txt'      #### 1095 images for validation


train_image_list = [line.rstrip('\n') for line in open(train_list_file)]
val_image_list = [line.rstrip('\n') for line in open(val_list_file)]

print(len(train_image_list), len(val_image_list))

### Notice a new data loader for siamese networks. This gives the image pairs (image_1, image_2) and a label as input to the siamese networks.
### see siamese_data_loader.py for details

trainloader = torch.utils.data.DataLoader(siamese_data_loader(img_root = img_root, image_list = train_list_file, crop=False,
                                                             resize = True, resize_shape=[128,128]), 
                                           batch_size=32, num_workers=1, shuffle = False, pin_memory=False)

testloader = torch.utils.data.DataLoader(siamese_data_loader(img_root = img_root, image_list = val_list_file, crop=False, mirror=False, 
                                                           resize = True, resize_shape=[128,128]), 
                                           batch_size=10, num_workers=1, shuffle = False, pin_memory=False)


classes = ['AamairKhan', 'Rimisen', 'Kajol', 'KareenaKapoor','RishiKapoor', 'AmrishPuri', 'AnilKapoor', 'AnupamKher', 'BomanIrani', 'HrithikRoshan', 'KajalAgarwal', 'KatrinaKaif', 'Madhavan', 'MadhuriDixit', 'Umashri', 'Trisha']

(5000, 1095)


In [0]:
for i in trainloader:
  print(i)

ERROR: couldn't find image ->  data/IMFDB_final/RishiKapoor/YehVadhaRaha/images/RishiKapoor_16.jpg
ERROR: couldn't find image ->  data/IMFDB_final/RishiKapoor/YehVadhaRaha/images/RishiKapoor_16.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Umashri/Nanjundi/images/Umashri_76.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Umashri/Nanjundi/images/Umashri_76.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/AnilKapoor/Misc/images/AnilKapoor_27.jpg
ERROR: couldn't find image ->  data/IMFDB_final/AnilKapoor/Misc/images/AnilKapoor_27.jpg
ERROR: couldn't find image ->  data/IMFDB_final/MadhuriDixit/Misc/images/MadhuriDixit_21.jpg
ERROR: couldn't find image ->  data/IMFDB_final/MadhuriDixit/Misc/images/MadhuriDixit_21.jpg
ERROR: couldn't find image ->  data/IMFDB_final/RishiKapoor/YehVadhaRaha/images/RishiKapoor_50.jpg
ERROR: couldn't find image ->  data/IMFDB_final/RishiKapoor/YehVadhaRaha/images/RishiKapoor_50.jpg
ERROR: couldn't find image ->  data/IMFDB

ERROR: couldn't find image ->  data/IMFDB_final/RishiKapoor/Bobby/images/RishiKapoor_29.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/RishiKapoor/Misc/images/RishiKapoor_102.jpg
ERROR: couldn't find image ->  data/IMFDB_final/RishiKapoor/Misc/images/RishiKapoor_102.jpg
ERROR: couldn't find image ->  data/IMFDB_final/AamairKhan/3Idiots/images/Amir_121.jpg
ERROR: couldn't find image ->  data/IMFDB_final/AamairKhan/3Idiots/images/Amir_121.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/KabhiKhushiKabhiGham/images/Kareenakapoor_143.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/KabhiKhushiKabhiGham/images/Kareenakapoor_143.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/MadhuriDixit/JamaiRaja/images/MadhuriDixit_15.jpg
ERROR: couldn't find image ->  data/IMFDB_final/MadhuriDixit/JamaiRaja/images/MadhuriDixit_15.jpg
creating a default image
ERROR: couldn't find image -> 

ERROR: couldn't find image ->  data/IMFDB_final/Rimisen/Misc/images/RimiSen_122.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/Chameli/images/Kareena_41.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/Chameli/images/Kareena_41.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KuchKuchhotahai/images/Kajol_95.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KuchKuchhotahai/images/Kajol_95.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Trisha/NamoVenkatesha/images/Trisha_211.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Trisha/NamoVenkatesha/images/Trisha_211.jpg
ERROR: couldn't find image ->  data/IMFDB_final/RishiKapoor/YehVadhaRaha/images/RishiKapoor_66.jpg
ERROR: couldn't find image ->  data/IMFDB_final/RishiKapoor/YehVadhaRaha/images/RishiKapoor_66.jpg
creating a default image
[
( 0 , 0 ,.,.) = 
   17   17   18  ...    40   43   49
   15   15   15  ...    40   43   46
   12   12   12  ...    42 

ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/KabhiKhushiKabhiGham/images/Kareenakapoor_129.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KatrinaKaif/ZindagiNaMilegiDobara/images/Katrinakaif_112.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KatrinaKaif/ZindagiNaMilegiDobara/images/Katrinakaif_112.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/AnilKapoor/JamaiRaja/images/AnilKapoor_223.jpg
ERROR: couldn't find image ->  data/IMFDB_final/AnilKapoor/JamaiRaja/images/AnilKapoor_223.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/AamairKhan/3Idiots/images/Amir_162.jpg
ERROR: couldn't find image ->  data/IMFDB_final/AamairKhan/3Idiots/images/Amir_162.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/AamairKhan/3Idiots/images/Amir_205.jpg
ERROR: couldn't find image ->  data/IMFDB_final/AamairKhan/3Idiots/images/Amir_205.jpg
ERROR: couldn't find image ->  data/IMFDB_final/AmrishPur

ERROR: couldn't find image ->  data/IMFDB_final/Madhavan/3Idiots/images/Madhavan_37.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Rimisen/Misc/images/RimiSen_152.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Rimisen/Misc/images/RimiSen_152.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KabhiKhushiKabhiGham/images/Kajol_49.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KabhiKhushiKabhiGham/images/Kajol_49.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KajalAgarwal/Misc/images/KajolAggarwal_168.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KajalAgarwal/Misc/images/KajolAggarwal_168.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/AnilKapoor/JamaiRaja/images/AnilKapoor_161.jpg
ERROR: couldn't find image ->  data/IMFDB_final/AnilKapoor/JamaiRaja/images/AnilKapoor_161.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/AnupamKher/Misc/images/AnupamKher_150.jpg
ERROR: couldn't find image ->  dat

...

ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/KabhiKhushiKabhiGham/images/Kareenakapoor_31.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/KabhiKhushiKabhiGham/images/Kareenakapoor_31.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KuchKuchhotahai/images/Kajol_54.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KuchKuchhotahai/images/Kajol_54.jpg
ERROR: couldn't find image ->  data/IMFDB_final/AamairKhan/AndazApnaApna/images/AamirKhan_263.jpg
ERROR: couldn't find image ->  data/IMFDB_final/AamairKhan/AndazApnaApna/images/AamirKhan_263.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KabhiKhushiKabhiGham/images/Kajol_100.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KabhiKhushiKabhiGham/images/Kajol_100.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KuchKuchhotahai/images/Kajol_161.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KuchKuchhotahai/images/Kajol_161.jpg
cre

ERROR: couldn't find image ->  data/IMFDB_final/HrithikRoshan/ZindagiNaMilegiDobara/images/HirtikRoshan_5.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/HrithikRoshan/ZindagiNaMilegiDobara/images/HirtikRoshan_106.jpg
ERROR: couldn't find image ->  data/IMFDB_final/HrithikRoshan/ZindagiNaMilegiDobara/images/HirtikRoshan_106.jpg
ERROR: couldn't find image ->  data/IMFDB_final/HrithikRoshan/ZindagiNaMilegiDobara/images/HirtikRoshan_167.jpg
ERROR: couldn't find image ->  data/IMFDB_final/HrithikRoshan/ZindagiNaMilegiDobara/images/HirtikRoshan_167.jpg
ERROR: couldn't find image ->  data/IMFDB_final/MadhuriDixit/JamaiRaja/images/MadhuriDixit_66.jpg
ERROR: couldn't find image ->  data/IMFDB_final/MadhuriDixit/JamaiRaja/images/MadhuriDixit_66.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/Trisha/NamoVenkatesha/images/Trisha_250.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Trisha/NamoVenkatesha/images/Trisha_250.jpg
ERROR: could

ERROR: couldn't find image ->  data/IMFDB_final/Rimisen/Misc/images/RimiSen_140.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/MadhuriDixit/Misc/images/MadhuriDixit_13.jpg
ERROR: couldn't find image ->  data/IMFDB_final/MadhuriDixit/Misc/images/MadhuriDixit_13.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Madhavan/3Idiots/images/Madhavan_23.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Madhavan/3Idiots/images/Madhavan_23.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/MadhuriDixit/HumApkeHaiKoun/images/MadhuriDixit_12.jpg
ERROR: couldn't find image ->  data/IMFDB_final/MadhuriDixit/HumApkeHaiKoun/images/MadhuriDixit_12.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/Chameli/images/Kareena_85.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/Chameli/images/Kareena_85.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KuchKuchhotahai/images/Kajol_198.jpg
ERROR: couldn't find im

ERROR: couldn't find image ->  data/IMFDB_final/MadhuriDixit/HumApkeHaiKoun/images/MadhuriDixit_15.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/KabhiKhushiKabhiGham/images/Kareenakapoor_98.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/KabhiKhushiKabhiGham/images/Kareenakapoor_98.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KuchKuchhotahai/images/Kajol_87.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Kajol/KuchKuchhotahai/images/Kajol_87.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/KabhiKhushiKabhiGham/images/Kareenakapoor_57.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/KabhiKhushiKabhiGham/images/Kareenakapoor_57.jpg
ERROR: couldn't find image ->  data/IMFDB_final/HrithikRoshan/KabhiKhushiKabhiGham/images/HrithikRoshan_148.jpg
ERROR: couldn't find image ->  data/IMFDB_final/HrithikRoshan/KabhiKhushiKabhiGham/images/HrithikRoshan_148.jpg

ERROR: couldn't find image ->  data/IMFDB_final/HrithikRoshan/ZindagiNaMilegiDobara/images/HirtikRoshan_219.jpg
ERROR: couldn't find image ->  data/IMFDB_final/HrithikRoshan/ZindagiNaMilegiDobara/images/HirtikRoshan_219.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/KabhiKhushiKabhiGham/images/Kareenakapoor_27.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KareenaKapoor/KabhiKhushiKabhiGham/images/Kareenakapoor_27.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/KatrinaKaif/ZindagiNaMilegiDobara/images/Katrinakaif_40.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KatrinaKaif/ZindagiNaMilegiDobara/images/Katrinakaif_40.jpg
creating a default image
ERROR: couldn't find image ->  data/IMFDB_final/KatrinaKaif/TeesMaarKhan/images/KatrinaKaif_50.jpg
ERROR: couldn't find image ->  data/IMFDB_final/KatrinaKaif/TeesMaarKhan/images/KatrinaKaif_50.jpg
ERROR: couldn't find image ->  data/IMFDB_final/AnilKapoor/JamaiRaja/images/AnilKapoor_29.

In [0]:
# Checking for GPU instance
use_cuda = torch.cuda.is_available()
#Intilizaing the loss value as high value
best_loss = 99999999

num_classes = 16

In [0]:
feature_net = network_9layers()   ### creates an object of this network architecture
feature_net = torch.load(cf.data_dir+'light_cnn/light_cnn_ckpt.t7')['net']


layers_to_remove = ['fc2']
for layers_ in layers_to_remove:        
    del(feature_net._modules[layers_])
    
classifier = nn.Sequential(nn.Linear(256, 64), nn.BatchNorm1d(64), nn.ReLU(),
                           nn.Linear(64, 32), nn.BatchNorm1d(32), nn.ReLU(),
                           nn.Linear(32, num_classes))

feature_net.fc2 = nn.Sequential(nn.Linear(256, 5))
if use_cuda:
    feature_net.cuda()
    classifier.cuda()
    


In [0]:
### Intiliazing the loss
criterion = nn.CrossEntropyLoss()
siamese_loss = contrastive_loss()   ### Notice a new loss. contrastive.py shows how to compute contrastive loss.

#### Lets train the siamese networks. The objective is images from same class (+ pair, label = 0) should have similar feature and images from different classes (- pair, label = 1) should have different features. Instead of having two physical networks sharing the weights, in implementation we have only one network and first pass image_1 (to get its feature) and then pass image_2 (to get its feature) through the same network. We then compute the contrastive loss on these feature pairs from input image pairs. This saves a lot of memory.

In [0]:
def train(epoch):
    print('\nEpoch: %d' % epoch)
    feature_net.train()
    train_loss = 0
    correct = 0
    total = 1
    for batch_idx, (inputs_1, inputs_2, targets) in enumerate(trainloader):
    
        if use_cuda:
            inputs_1, inputs_2, targets = inputs_1.cuda(), inputs_2.cuda(), targets.cuda()
        optimizer.zero_grad()
        inputs_1, inputs_2, targets = Variable(inputs_1), Variable(inputs_2), Variable(targets)
        features_1 = feature_net.get_features2(inputs_1)      ### get feature for image_1
        features_2 = feature_net.get_features2(inputs_2)      ### get feature for image_2
        
        loss = siamese_loss(features_1, features_2, targets.float())   ### compute the contrastive loss, computes the similarity between the features.
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        
        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f '
                         % (train_loss/(batch_idx+1)))
        
    train_loss_file.write('%d %.3f %.3f\n' %(epoch, train_loss/len(trainloader), 100.*correct/total))



In [0]:
def test(epoch):
    global best_loss
    feature_net.eval()
    test_loss = 0
    correct = 0
    total = 1
    for batch_idx, (inputs_1, inputs_2, targets) in enumerate(testloader):
        if use_cuda:
            inputs_1, inputs_2, targets = inputs_1.cuda(), inputs_2.cuda(), targets.cuda()
        optimizer.zero_grad()
        inputs_1, inputs_2, targets = Variable(inputs_1), Variable(inputs_2), Variable(targets)
        features_1 = feature_net.get_features2(inputs_1)      
        features_2 = feature_net.get_features2(inputs_2)      
        
        loss = siamese_loss(features_1, features_2, targets.float())
        test_loss += loss.data[0]
        
        progress_bar(batch_idx, len(testloader), 'Loss: %.3f '
                         % (test_loss/(batch_idx+1)))
        
    val_loss_file.write('%d %.3f %.3f\n' %(epoch,  test_loss/len(testloader), 100.*correct/total))

    # Save checkpoint.
    losss = test_loss/len(testloader)
    if  losss < best_loss:   ### save model with the best loss so far
        print('Saving..') 
        state = {
            'net': feature_net
        }
        if not os.path.isdir(cf.data_dir+'checkpoint'):
            os.mkdir(cf.data_dir+'checkpoint')
        torch.save(state, cf.data_dir+'checkpoint/siamese_ckpt.t7')
        best_loss = losss
    
    return test_loss/len(testloader)

In [0]:
experiment = 'siamese_IMFDB'
train_loss_file = open(cf.data_dir+experiment+"train_loss.txt", "w", 0)
val_loss_file = open(cf.data_dir+experiment+"val_loss.txt", "w", 0)

In [12]:
optimizer = optim.Adam(feature_net.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True)   #### dynamic LR scheduler
for epoch in range(0, 1):
    train(epoch)
    test_loss = test(epoch)
    scheduler.step(test_loss)
    
train_loss_file.close()
val_loss_file.close()


Epoch: 0
ERROR: couldn't find image ->  data/IMFDB_final/Madhavan/MydearMunnabhai/images/Madhavan_10.jpg
ERROR: couldn't find image ->  data/IMFDB_final/Madhavan/MydearMunnabhai/images/Madhavan_10.jpg


error: ignored

In [0]:
### After training we load the model that performed the best on validation data (avoid picking overfitted model)
### we will use the base pre-trained network for feature extraction only. This feature is used to train an MLP classifier.

feature_net = torch.load(cf.data_dir+'checkpoint/siamese_ckpt.t7')['net'].eval()

##### Lets see how well does the siamese detect an imposter. We check whether image_2 is same individual as image_1 or an imposter. We do this by computing dissimilarity score between features.

In [0]:
testloader = torch.utils.data.DataLoader(siamese_data_loader(img_root = img_root, image_list = val_list_file, crop=False, mirror=False, 
                                                           resize = True, resize_shape=[128,128]), 
                                           batch_size=1, num_workers=1, shuffle = False, pin_memory=False)

lab = ['same', 'imposter']
for batch_idx, (inputs_1, inputs_2, targets) in enumerate(testloader):
    if batch_idx%10 == 0 or int(targets)==0:      ### show every tenth image or if its the same individual
        
        if use_cuda:
            inputs_1, inputs_2, targets = inputs_1.cuda(), inputs_2.cuda(), targets.cuda()
        
        inputs_1, inputs_2, targets = Variable(inputs_1), Variable(inputs_2), Variable(targets)
        features_1 = feature_net.get_features2(inputs_1)      
        features_2 = feature_net.get_features2(inputs_2)      
        
        dissimilarity = F.pairwise_distance(features_1, features_2).data[0].cpu().numpy()[0]
    
        img = np.concatenate((inputs_1.data.cpu().numpy()[0][0], inputs_2.data.cpu().numpy()[0][0]), axis = 1)
        plt.imshow(img, cmap='gray')
        plt.text(100,20,str(dissimilarity), fontsize=24)     ### similarity score
        plt.text(100,40,lab[int(targets.data[0])], fontsize=24)   ### ground truth
        plt.show()
    

### Now we use this network for feature extraction and train an MLP classifier. Feature_net is not updated/train/tweak after this. We only train the MLP classifier.

In [0]:
from data_loader import custom_data_loader

train_list_file = cf.data_dir+'IMFDB_train.txt'   #### 5000 images for training
val_list_file = cf.data_dir+'IMFDB_test.txt'      #### 1095 images for validation


train_image_list = [line.rstrip('\n') for line in open(train_list_file)]
val_image_list = [line.rstrip('\n') for line in open(val_list_file)]

print(len(train_image_list), len(val_image_list))

trainloader = torch.utils.data.DataLoader(custom_data_loader(img_root = img_root, image_list = train_list_file, crop=False,
                                                             resize = True, resize_shape=[128,128]), 
                                          batch_size=32, num_workers=16, shuffle = True, pin_memory=False)

testloader = torch.utils.data.DataLoader(custom_data_loader(img_root = img_root, image_list = val_list_file, crop=False, mirror=False, 
                                                           resize = True, resize_shape=[128,128]), 
                                         batch_size=10, num_workers=5, shuffle = False, pin_memory=False)

In [0]:
def train_classifier(epoch):
    print('\nEpoch: %d' % epoch)
    classifier.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(trainloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets)
        features = feature_net.get_features(inputs).detach()       
        
        
        outputs = classifier(features)
        size_ = outputs.size()
        outputs_ = outputs.view(size_[0], num_classes)
        loss = criterion(outputs_, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        _, predicted = torch.max(outputs_.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        
        progress_bar(batch_idx, len(trainloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (train_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
    train_loss_file.write('%d %.3f %.3f\n' %(epoch, train_loss/len(trainloader), 100.*correct/total))


In [0]:
def test_classifier(epoch):
    global best_acc
    classifier.eval()
    test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(testloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        features = feature_net.get_features(inputs).detach()
        
        outputs = classifier(features)
        size_ = outputs.size()
        outputs_ = outputs.view(size_[0], num_classes)
        loss = criterion(outputs_, targets)

        test_loss += loss.data[0]
        _, predicted = torch.max(outputs_.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        
        progress_bar(batch_idx, len(testloader), 'Loss: %.3f | Acc: %.3f%% (%d/%d)'
                         % (test_loss/(batch_idx+1), 100.*correct/total, correct, total))
        
    val_loss_file.write('%d %.3f %.3f\n' %(epoch,  test_loss/len(testloader), 100.*correct/total))

    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': classifier,
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir(cf.data_dir+'checkpoint'):
            os.mkdir(cf.data_dir+'checkpoint')
        torch.save(state, cf.data_dir+'checkpoint/checkpoint_ckpt.t7')
        best_acc = acc
    
    return test_loss/len(testloader)

In [0]:
best_acc = 0
experiment = 'siamese_features_IMFDB'
train_loss_file = open(cf.data_dir+experiment+"train_loss.txt", "w", 0)
val_loss_file = open(cf.data_dir+experiment+"val_loss.txt", "w", 0)

In [0]:
optimizer = optim.Adam(classifier.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.5, patience=2, verbose=True)   #### dynamic LR scheduler
for epoch in range(0, 30):
    train_classifier(epoch)
    test_loss = test_classifier(epoch)
    scheduler.step(test_loss)
    
train_loss_file.close()
val_loss_file.close()

In [0]:
def eval():
    feature_net.eval()
    classifier.eval()
    
    testloader = torch.utils.data.DataLoader(custom_data_loader(img_root = img_root, image_list = val_list_file, crop=False, mirror=False, 
                                                           resize = True, resize_shape=[128,128]), 
                                           batch_size=1, num_workers=1, shuffle = False, pin_memory=False)
    correct = 0
    total = 0
    conf_mat = np.zeros((num_classes, num_classes))
    total_ = 1e-12+np.zeros((num_classes))
    wrong_predictions = []
    for batch_idx, (inputs, targets) in enumerate(testloader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        inputs, targets = Variable(inputs, volatile=True), Variable(targets)
        features = feature_net.get_features(inputs).detach()
        
        outputs = classifier(features)
        size_ = outputs.size()
        outputs_ = outputs.view(size_[0], num_classes)
        _, predicted = torch.max(outputs_.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()
        prediction = predicted.cpu().numpy()[0]
        targets = targets.data.cpu().numpy()[0]
        total_[targets] +=1
        conf_mat[predicted, targets] +=1
        
        if prediction != targets:
            wrong_predictions += [[inputs, prediction, targets]]
        
    for k in range(num_classes):
        conf_mat[:,k] /= total_[k]
    return conf_mat, 100.*correct/total, wrong_predictions
    

In [0]:
conf, acc, wrong_predictions = eval()
print(acc)

In [0]:
plt.imshow(conf, cmap='jet', vmin=0, vmax = 1)
plt.show()

In [0]:
for w in wrong_predictions[::10]:
    print(classes[w[2]], 'confused with', classes[w[1]])
    plt.imshow(w[0][0][0].data.cpu().numpy(), cmap='gray')
    plt.show()